In [2]:
import os
import numpy as np
import pandas as pd
import re

os.chdir('C:/Users/tup54227/Documents/GitHub/istart/Shared_Reward')
#Make a list of all the SR Files
SR_flist=[os.path.join(root, f) for root, dirs, files in os.walk('logs') for f in files if 'SR-Rating' in f]

#Make a list of Dataframes
ratings_list=[]
for f in SR_flist:
    sub=re.search('sub(.*)_',f).group(1)
    if any(i.isdigit() for i in sub):
        tmp_df=pd.read_csv(f)
        tmp_df['sub']=sub
        ratings_list.append(tmp_df)
#Concatonate the DataFrames together
srpr_df=pd.concat(ratings_list)
srpr_df=srpr_df.reset_index(drop=True)
srpr_df['Rating'].astype(int)

ValueError: invalid literal for int() with base 10: '0.0'

In [1]:
Trait = {0:'Win', 1:'Lose'}
Partner = {3:'Friend',2:'Stranger',1:'Comp'}

NameError: name 'srpr_df' is not defined

In [ ]:
#This loop collapses the 6 rows of Shared Reward Partner Ratings into 1 row for each participant
collapsed = {}
for index, row in srpr_df.iterrows():
    if(not row['sub'] in collapsed):
        collapsed[row['sub']] = {}
    column_name = Trait[row['Trait']] + '-' + Partner[row['Partner']]
    collapsed[row['sub']][column_name] = row['Rating']

In [ ]:
#PR = Partner Ratings from post-scan task
PR = pd.DataFrame(collapsed).T
PR = PR.reset_index().rename(columns = {'index':'sub'})
PR.to_csv('SR_PartnerRatings.csv',index=False)
pd.read_csv('SR_PartnerRatings.csv')

In [ ]:
#RC = REDCap data
RC = pd.read_csv('SR_REDCap_data.csv').rename(columns = {'participant_id':'sub', 'ios_p':'closeness_rating'})
#RS = Reward Sensitivity scores
RS = pd.read_csv('Reward_Sensitivity.csv').rename(columns = {'RealID':'sub'})
#ASRD = All Shared Reward Data
ASRD = RS.merge(PR, on = 'sub', how = 'right').sort_values('sub', ascending = True, ignore_index = True)
ASRD = ASRD.merge(RC, on = 'sub', how = 'left').sort_values('sub', ascending = True, ignore_index = True)
ASRD.drop('redcap_event_name', axis = 'columns').to_csv('AllSharedRewardData.csv', index = False)
pd.read_csv('AllSharedRewardData.csv')